In [1]:
import numpy as np
import pandas as pd
import sys
import json
import pickle5 as pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download("punkt")

from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ahmetsenturk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ahmetsenturk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello world!", return_tensors="pt")
inputs
# outputs = model(**inputs)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [4]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('bert-base-nli-mean-tokens')


  0%|          | 0.00/405M [00:00<?, ?B/s]
  0%|          | 65.5k/405M [00:00<10:24, 649kB/s]
  0%|          | 147k/405M [00:00<10:02, 672kB/s] 
  0%|          | 262k/405M [00:00<09:06, 740kB/s]
  0%|          | 360k/405M [00:00<09:36, 702kB/s]
  0%|          | 475k/405M [00:00<08:06, 833kB/s]
  0%|          | 562k/405M [00:00<08:57, 752kB/s]
  0%|          | 754k/405M [00:00<06:18, 1.07MB/s]
  0%|          | 918k/405M [00:00<05:47, 1.16MB/s]
  0%|          | 1.13M/405M [00:01<04:42, 1.43MB/s]
  0%|          | 1.33M/405M [00:01<04:31, 1.49MB/s]
  0%|          | 1.51M/405M [00:01<04:25, 1.52MB/s]
  0%|          | 1.72M/405M [00:01<04:20, 1.55MB/s]
  0%|          | 1.93M/405M [00:01<04:00, 1.67MB/s]
  1%|          | 2.10M/405M [00:01<04:14, 1.58MB/s]
  1%|          | 2.28M/405M [00:01<04:21, 1.54MB/s]
  1%|          | 2.51M/405M [00:01<04:01, 1.67MB/s]
  1%|          | 2.72M/405M [00:02<03:55, 1.71MB/s]
  1%|          | 2.98M/405M [00:02<03:38, 1.84MB/s]
  1%|          | 3.23M/405M [00:

 32%|███▏      | 128M/405M [00:17<00:28, 9.58MB/s]
 32%|███▏      | 129M/405M [00:17<00:27, 9.93MB/s]
 32%|███▏      | 130M/405M [00:17<00:26, 10.2MB/s]
 32%|███▏      | 131M/405M [00:17<00:28, 9.56MB/s]
 33%|███▎      | 132M/405M [00:17<00:27, 9.86MB/s]
 33%|███▎      | 133M/405M [00:17<00:27, 9.89MB/s]
 33%|███▎      | 134M/405M [00:17<00:28, 9.35MB/s]
 33%|███▎      | 135M/405M [00:18<00:29, 9.04MB/s]
 34%|███▎      | 136M/405M [00:18<00:28, 9.52MB/s]
 34%|███▍      | 137M/405M [00:18<00:29, 9.22MB/s]
 34%|███▍      | 138M/405M [00:18<00:32, 8.10MB/s]
 34%|███▍      | 139M/405M [00:18<00:32, 8.27MB/s]
 35%|███▍      | 140M/405M [00:18<00:30, 8.64MB/s]
 35%|███▍      | 141M/405M [00:18<00:28, 9.21MB/s]
 35%|███▌      | 142M/405M [00:18<00:27, 9.50MB/s]
 35%|███▌      | 143M/405M [00:18<00:26, 9.87MB/s]
 36%|███▌      | 144M/405M [00:19<00:28, 9.27MB/s]
 36%|███▌      | 145M/405M [00:19<00:29, 8.91MB/s]
 36%|███▌      | 146M/405M [00:19<00:28, 9.15MB/s]
 36%|███▋      | 147M/405M [00:

 68%|██████▊   | 274M/405M [00:35<00:59, 2.23MB/s]
 68%|██████▊   | 274M/405M [00:35<01:03, 2.07MB/s]
 68%|██████▊   | 274M/405M [00:35<01:04, 2.03MB/s]
 68%|██████▊   | 274M/405M [00:35<01:05, 2.00MB/s]
 68%|██████▊   | 275M/405M [00:36<01:05, 1.98MB/s]
 68%|██████▊   | 275M/405M [00:36<01:04, 2.03MB/s]
 68%|██████▊   | 275M/405M [00:36<01:06, 1.94MB/s]
 68%|██████▊   | 275M/405M [00:36<01:07, 1.92MB/s]
 68%|██████▊   | 275M/405M [00:36<01:06, 1.95MB/s]
 68%|██████▊   | 276M/405M [00:36<01:16, 1.69MB/s]
 68%|██████▊   | 276M/405M [00:36<01:14, 1.74MB/s]
 68%|██████▊   | 276M/405M [00:36<01:06, 1.93MB/s]
 68%|██████▊   | 277M/405M [00:37<00:55, 2.32MB/s]
 68%|██████▊   | 277M/405M [00:37<00:49, 2.58MB/s]
 68%|██████▊   | 277M/405M [00:37<00:45, 2.80MB/s]
 69%|██████▊   | 278M/405M [00:37<00:45, 2.82MB/s]
 69%|██████▊   | 278M/405M [00:37<00:48, 2.60MB/s]
 69%|██████▊   | 278M/405M [00:37<00:44, 2.82MB/s]
 69%|██████▊   | 279M/405M [00:37<00:49, 2.57MB/s]
 69%|██████▉   | 279M/405M [00:

 78%|███████▊  | 317M/405M [00:53<00:35, 2.53MB/s]
 78%|███████▊  | 317M/405M [00:53<00:35, 2.48MB/s]
 78%|███████▊  | 317M/405M [00:53<00:36, 2.42MB/s]
 78%|███████▊  | 317M/405M [00:53<00:37, 2.37MB/s]
 78%|███████▊  | 318M/405M [00:53<00:37, 2.34MB/s]
 78%|███████▊  | 318M/405M [00:53<00:35, 2.44MB/s]
 79%|███████▊  | 318M/405M [00:53<00:40, 2.14MB/s]
 79%|███████▊  | 318M/405M [00:53<00:36, 2.40MB/s]
 79%|███████▊  | 319M/405M [00:54<00:34, 2.50MB/s]
 79%|███████▊  | 319M/405M [00:54<00:34, 2.50MB/s]
 79%|███████▉  | 319M/405M [00:54<00:35, 2.42MB/s]
 79%|███████▉  | 320M/405M [00:54<00:34, 2.49MB/s]
 79%|███████▉  | 320M/405M [00:54<00:31, 2.69MB/s]
 79%|███████▉  | 320M/405M [00:54<00:32, 2.61MB/s]
 79%|███████▉  | 321M/405M [00:54<00:31, 2.73MB/s]
 79%|███████▉  | 321M/405M [00:54<00:32, 2.61MB/s]
 79%|███████▉  | 321M/405M [00:54<00:32, 2.60MB/s]
 79%|███████▉  | 321M/405M [00:55<00:31, 2.64MB/s]
 79%|███████▉  | 322M/405M [00:55<00:30, 2.73MB/s]
 79%|███████▉  | 322M/405M [00:

 90%|█████████ | 365M/405M [01:11<00:19, 1.99MB/s]
 90%|█████████ | 366M/405M [01:11<00:19, 1.99MB/s]
 90%|█████████ | 366M/405M [01:11<00:19, 2.01MB/s]
 90%|█████████ | 366M/405M [01:11<00:19, 1.98MB/s]
 90%|█████████ | 366M/405M [01:11<00:20, 1.88MB/s]
 90%|█████████ | 367M/405M [01:11<00:20, 1.93MB/s]
 91%|█████████ | 367M/405M [01:11<00:20, 1.90MB/s]
 91%|█████████ | 367M/405M [01:11<00:19, 2.00MB/s]
 91%|█████████ | 367M/405M [01:12<00:19, 1.95MB/s]
 91%|█████████ | 368M/405M [01:12<00:17, 2.19MB/s]
 91%|█████████ | 368M/405M [01:12<00:17, 2.16MB/s]
 91%|█████████ | 368M/405M [01:12<00:17, 2.11MB/s]
 91%|█████████ | 368M/405M [01:12<00:18, 2.00MB/s]
 91%|█████████ | 368M/405M [01:12<00:17, 2.08MB/s]
 91%|█████████ | 369M/405M [01:12<00:17, 2.10MB/s]
 91%|█████████ | 369M/405M [01:12<00:18, 1.94MB/s]
 91%|█████████ | 369M/405M [01:12<00:17, 2.09MB/s]
 91%|█████████ | 369M/405M [01:13<00:16, 2.16MB/s]
 91%|█████████ | 370M/405M [01:13<00:18, 1.90MB/s]
 91%|█████████▏| 370M/405M [01:

In [6]:
topics_file = open("topics.pickle", "rb")
all_documents_file = open("all_documents.pickle", "rb")

In [7]:
topics_dict = pickle.load(topics_file)
all_docs_dict = pickle.load(all_documents_file)

In [8]:
topics_df = pd.DataFrame(topics_dict)
all_docs_df = pd.DataFrame(all_docs_dict)

In [9]:
all_docs_df.drop_duplicates(subset=['document_id'], keep=False, inplace=True)

In [10]:
all_docs_df["text"] = all_docs_df["abstract"] + " " + all_docs_df["title"] 

In [19]:
stop_words = set(stopwords.words('english')) 
tokenizer = RegexpTokenizer(r'\w+')
all_docs_splitted = []
for text in all_docs_df["text"]:
    if text != "":
        filtered_text = ""
        for t in tokenizer.tokenize(text):
            if t not in stop_words:
                filtered_text = filtered_text + " " + t
        all_docs_splitted.append(filtered_text)

In [20]:
topics_df["text"] = topics_df["query"] + " " + topics_df["question"] + " " + topics_df["narrative"] 

In [21]:
topics_splitted = []
for text in topics_df["text"]:
    if text != "":
        filtered_text = ""
        for t in tokenizer.tokenize(text):
            if t not in stop_words:
                filtered_text = filtered_text + " " + t
        topics_splitted.append(filtered_text)

In [ ]:
corpus_embeddings = embedder.encode(all_docs_splitted)

In [ ]:
query_embeddings = embedder.encode(topics_splitted)

In [82]:
bm25 = BM25Okapi(all_docs_splitted)

In [87]:
results = pd.DataFrame()
train_results = pd.DataFrame()
test_results = pd.DataFrame()
for topic_index in range(0, len(topics_df.index)):
    query = topics_df.iloc[topic_index]["query"] + " " + topics_df.iloc[topic_index]["question"] + " " + topics_df.iloc[topic_index]["narrative"]
    filtered_query = []
    if query != "":
        for t in tokenizer.tokenize(query):
            if t not in stop_words:
                filtered_query.append(t.lower())
    
    query_results = bm25.get_scores(filtered_query)
    query_results_df = pd.DataFrame(query_results)
    query_results_df.columns = ["bm25 results"]
    query_results_df.insert(0, "topic_id", [(topic_index + 1) for x in range(0, len(query_results_df.index))])
    query_results_df.insert(1, "temp1", ["Q0" for x in range(0, len(query_results_df.index))])
    query_results_df.insert(2, "doc_id", [all_docs_df.iloc[i,0] for i in range(0, len(query_results_df.index))])
    query_results_df.insert(3, "rank", ["0" for x in range(0, len(query_results_df.index))])
    query_results_df["standard"] = "STANDARD"
    results = pd.concat([results, query_results_df], axis=0) 
    if topic_index % 2 == 0:
        test_results = pd.concat([train_results, query_results_df], axis=0) 
    else:
        train_results = pd.concat([test_results, query_results_df], axis=0)    
    

In [88]:
results

,topic_id,temp1,doc_id,rank,bm25 results,standard
0,1,Q0,ug7v899j,0,0.988723,STANDARD
1,1,Q0,02tnwd4m,0,0.000000,STANDARD
2,1,Q0,ejv2xln0,0,0.000000,STANDARD
3,1,Q0,2b73a28n,0,0.000000,STANDARD
4,1,Q0,9785vg6d,0,9.386451,STANDARD
...,...,...,...,...,...,...
189975,50,Q0,z4ro6lmh,0,1.559466,STANDARD
189976,50,Q0,hi8k8wvb,0,7.731445,STANDARD
189977,50,Q0,ma3ndg41,0,1.571896,STANDARD
189978,50,Q0,wh10285j,0,1.548744,STANDARD


In [89]:
np.savetxt("./results.txt", results.values, fmt="%s")

In [90]:
np.savetxt("./test_results.txt", test_results.values, fmt="%s")
np.savetxt("./train_results.txt", train_results.values, fmt="%s")